In [20]:
from djitellopy import Tello
import numpy as np
import pandas as pd
import cv2
from tensorflow.keras.models import load_model
import mediapipe as mp

In [21]:
import tensorflow as tf

In [22]:


model = load_model('model.h5')#loading our ml model, for predicting hand gestures
# tello=Tello()
# def droneaction(gesture):
#     if (gesture=='Forward'):
#         tello.move_forward(100)
#     elif(gesture=='Backward'):
#         tello.move_back(100)
#     elif(gesture=='Left'):
#         tello.move_left(100)
#     elif(gesture=='Right'):
#         tello.move_right(100)
#     elif(gesture=='Up'):
#         tello.move_up(100)
#     elif(gesture=='Down'):
#         tello.move_down(100)
#     elif(gesture=='Stop'):
#         tello.rotate_counter_clockwise(180)
#     else:
#         pass
# try:
#     tello.connect() #connects to drone
# except Exception as e:
#     print(e)
# try:
#     tello.takeoff()#drone takesoff
# except Exception as e:
#     print(e)

In [19]:
mpHands = mp.solutions.hands
hands = mpHands.Hands()
mpDraw = mp.solutions.drawing_utils
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 500) 
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 500)
if not cap.isOpened():
        print("Error: Could not open webcam.")
count=0
prediction=''
current=''
labels={0:'Stop',1:'Backward',2:'Forward',3:'Up',4:'Down',5:'Left',6:'Right',7:'Flip'}
while True:  # Continuous frames
    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to read frame.")
        break
        
    frame = cv2.flip(frame, 1) 
    y, x, channel = frame.shape
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(img)  
    landmarks=[]

    if result.multi_hand_landmarks:  # we will get a list of dictionaries with each landmark as key and x,y coords as value
        for handslms in result.multi_hand_landmarks:
            mpDraw.draw_landmarks(frame, handslms, mpHands.HAND_CONNECTIONS)

            lndmarks = handslms.landmark
            for i in range(len(lndmarks)):#it has only 2 values one x and onr y
                landmarks.append(lndmarks[i].x*x)
                landmarks.append(lndmarks[i].y*y)
        try:
            confidence_array=model.predict(tf.expand_dims(landmarks,axis=0))#converts to 2d array for input
            print(confidence_array)
        except Exception as e:
            print(e)
        classtype=np.argmax(confidence_array)
        if(confidence_array[0][classtype]>0.8):
            gesture=labels[classtype]
            if(gesture==current):
                count+=1
            else:
                current=gesture
                count=0
            if(count==5):
                prediction=gesture
                count=0
                # droneaction(prediction)

        cv2.putText(frame, prediction, (250,250), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), lineType=cv2.LINE_AA)



        
            
    cv2.imshow("AbhinavFTW", frame)
    key = cv2.waitKey(1)
        
    if key == ord("q"):  # Breaks input
        break
   
    
cap.release()
cv2.destroyAllWindows()
# tello.end()


I0000 00:00:1737234726.350932  420526 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M1
W0000 00:00:1737234726.367016  468143 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1737234726.382586  468143 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
[[1.2632109e-04 4.1939606e-07 2.6609835e-01 1.2790498e-04 1.6913344e-05
  6.4154357e-01 3.5037076e-05 9.2051461e-02]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
[[3.1944896e-07 2.5867232e-12 9.3781973e-06 9.7556779e-08 5.8138010e-09
  9.9820924e-01 1.2391172e-07 1.7807666e-03]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
[[1.6709862e-07 4.7576943e-13 5.9432905e-06 1.2768197e-08 2.1846893e-09
  9.9975330e-01 1.8271084e-08 2.4062835e-04]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
[[7.5950652e-07 1.7483397e-11 4.5036199e-05 1.4681305e-07 3.7462417e-08
  9.9926239e-01 8.5717659e-08 6.9145259e-04]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
[[4.4392635e-07 5.7661931e-12 1.4326956e-05 8.4613568e-08 1.5405719e-08
  9.9936527e-01 6.6201977e-08 6.1980257e-04]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
[[1.2340697e-06 4.2700964e-11 6.3250569e-05 2.2338433e-07 8.8286164e-08
  9.9937552e-01 1.1420525e-07 5.5964640e-04]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
[[1.7006557e-06 9.4456630